This Notebook contains :


1.   Importing configuration file
2.   Importing essential information from config file
3.   General Crawling which will run daily to capture daily landslide events and focused crawling which will run on occasional basis to capture more videos related to any particular events. ( Both for twitter and youtube )
4.   Saved General crawler CSV on google drive folder

Note : This notebook was automatically ran on daily basis with the help of deepnote.com



Adding all the dependencies


In [ ]:
%%capture
!pip3 install google-api-python-client
!pip install apiclient
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
#!pip3 install google-api-python-client
#from apiclient.discovery import build
#from apiclient import discovery
#from discovery import build
from googleapiclient.discovery import build
import argparse
import csv
import pandas as pd
from sklearn.cluster import KMeans
import datetime as dt
from matplotlib import pyplot as plt
import math

Importing Configuration File

---
Reference : https://www.kite.com/python/answers/how-to-update-a-json-file-in-python


In [ ]:
# Listing Datasets which are being saved in google drive
!ls /datasets/outputyoutube/Landslide_Project

Cleaning_Clustering_Youtube_Landslide_Prediction.ipynb
config_.json
India_Locations_Villages
Scraping_Youtube_Twitter.ipynb
Twitter_General_July01.csv
Twitter_General_July02.csv
Twitter_General_July03.csv
Twitter_General_July04.csv
Twitter_General_July05.csv
Twitter_General_July06.csv
Twitter_General_July07.csv
Twitter_General_July08.csv
Twitter_General_July09.csv
Twitter_General_June13.csv
Twitter_General_June15.csv
Twitter_General_June16.csv
Twitter_General_June17.csv
Twitter_General_June18.csv
Twitter_General_June19.csv
Twitter_General_June20.csv
Twitter_General_June21.csv
Twitter_General_June22.csv
Twitter_General_June23.csv
Twitter_General_June24.csv
Twitter_General_June25.csv
Twitter_General_June26.csv
Twitter_General_June27.csv
Twitter_General_June28.csv
Twitter_General_June29.csv
Twitter_General_June30.csv
YouTube
Youtube_General_July01.csv
Youtube_General_July02.csv
Youtube_General_July03.csv
Youtube_General_July04.csv
Youtube_General_July05.csv
Youtube_General_July06.csv
Yout

In [ ]:
#!ls /datasets/configuration-file

In [ ]:
import json
a_file = open("/datasets/outputyoutube/Landslide_Project/config_.json","r+",encoding='UTF-8')
json_object = json.load(a_file)
a_file.close()


#Collecting Data From Youtube


In [ ]:
# Please register and replace this with your key
# My API KEY : 
# MY API KEY 2 : 
DEVELOPER_KEY = ""
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [ ]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

## Generic Crawling






In [ ]:
# Parsing the list of General Keywords
Gen_Keywords =json_object["General-Crawling"]["General-Keywords"]
Gen_Keywords=list(Gen_Keywords)[0]
keys = f"{Gen_Keywords}"
Gen_keywords=Gen_Keywords.split(',')
# Gen_keywords

In [ ]:
%%capture
!pip install youtube_transcript_api
!pip install google-api-python-client

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
import datetime
import pandas as pd
import time
from datetime import timedelta

In [ ]:
from datetime import date
curr_time = datetime.datetime.now()
#print(curr_time)
published_after = curr_time - datetime.timedelta(days=1) + datetime.timedelta(minutes=5)
published_after=published_after.strftime("%Y-%m-%dT%XZ") # converting into youtube api format
print("The General Crawler will start Scrapping from "+published_after)

The General Crawler will start Scrapping from 2021-07-13T05:01:56Z


In [1]:
# If we want to publish alternate to daily basis we can edit in config file and uncomment this code
# published_after_alt=json_object['General-Crawling']['General-Keywords'][keys]['End_Datetime']

In [ ]:
# Reference : https://www.pragnakalp.com/automate-youtube-video-transcription-python/

column_names=["Topic", "Channel_Name" ,"TimeQueried", "Video_Title", "Video_ID", "Published_At", "duration", "viewCount", "likeCount", "dislikeCount",
      "favoriteCount","commentCount", "Description","Location","Tags","Latitude","Longitude"]
ddf2=pd.DataFrame(columns=column_names)
textlist=[]
for Query in Gen_keywords:
  videos=[]
  next_page_token = None
  while 1:
    search_response = youtube.search().list(q=Query,type="video", part="id,snippet", maxResults=100,regionCode="IN",order="relevance",publishedAfter=published_after).execute() 
    videos+=search_response['items']
    #next_page_token = search_response.get('nextPageToken')
    #print(next_page_token)
    if next_page_token is None:
            break
  count=0
  video_ids=[]
  for video in videos:
      video_ids.append(video['id']['videoId'])
      count=count+1
  count 
  #text_df = pd.DataFrame(textlist,columns=["Transcript"])
  for video_id in video_ids:
      templist=[]
      try:
        responses = YouTubeTranscriptApi.get_transcript(
            video_id, languages=['en'])
        #responses2=responses.get_transcript
        for response in responses:
            text = response['text']
            templist.append(text)
        textlist.append(templist)
      except Exception as e:
        textlist.append("NaN") # Generally when transcript are not available or too many requests are there

  for k in range(len(video_ids)):
    request = youtube.videos().list(
            part="statistics,contentDetails,snippet,status,recordingDetails",
            id=video_ids[k]
        )
    response = request.execute()
    #print(response)
    #print(response["items"][0]["statistics"]["likeCount"])
    #print(response["items"][0]["snippet"]["channelTitle"])
    #print(response["items"][0]["snippet"]['tags'][0:10])
    #print(response['items'][0])
    # try :
    #   print(response["items"][0]["recordingDetails"]['location']['latitude'])
    # except :
    #   print("no location")
    #print(response["items"][0]["snippet"]["description"])
    #response["items"][0]["statistics"]["favoriteCount"]

    try:
        ddf2 = ddf2.append({"Topic":Query,"Channel_Name":response["items"][0]["snippet"]["channelTitle"],"TimeQueried":dt.datetime.now(), "Video_Title":response["items"][0]["snippet"]["title"], "Video_ID":response["items"][0]["id"], "Published_At":response["items"][0]["snippet"]["publishedAt"], "duration":response["items"][0]["contentDetails"]["duration"], "viewCount":response["items"][0]["statistics"]["viewCount"], "likeCount":response["items"][0]["statistics"]["likeCount"], "dislikeCount":response["items"][0]["statistics"]["dislikeCount"],"favoriteCount":response["items"][0]["statistics"]["favoriteCount"],"commentCount":response["items"][0]["statistics"]["commentCount"], "Description":response["items"][0]["snippet"]["description"],"Location": response["items"][0]["recordingDetails"]["location"],"Tags":response["items"][0]["snippet"]['tags'][0:10],"Latitude":response["items"][0]["recordingDetails"]['location']['latitude'],"Longitude":response["items"][0]["recordingDetails"]['location']['longitude']},ignore_index = True)

    except:
        ddf2 = ddf2.append({"Topic":Query,"Channel_Name":response["items"][0]["snippet"]["channelTitle"] ,"TimeQueried":dt.datetime.now(), "Video_Title":response["items"][0]["snippet"]["title"], "Video_ID":response["items"][0]["id"], "Published_At":response["items"][0]["snippet"]["publishedAt"], "duration":response["items"][0]["contentDetails"]["duration"], "viewCount":response["items"][0]["statistics"]["viewCount"], "likeCount":"", "dislikeCount":"","favoriteCount":"","commentCount":"", "Description":response["items"][0]["snippet"]["description"],"Tags":"","Latitude":"","Longitude":""},ignore_index = True)

In [ ]:
# ddf2.shape

In [ ]:
# ddf2.head(10)

In [ ]:
text_df = pd.DataFrame()
text_df["Transcript"] = textlist

In [ ]:
# text_df.shape

In [ ]:
ddf2 = pd.concat([ddf2,text_df],axis=1,ignore_index=False,join="outer")

In [ ]:
#ddf2.head()

In [ ]:
#ddf2['Transcript'].unique()

In [ ]:
ddf2=ddf2.drop_duplicates(subset=['Video_ID',
                                  'Video_Title'])

In [ ]:
#ddf2.shape

In [ ]:
ddf2.head()

,Topic,Channel_Name,TimeQueried,Video_Title,Video_ID,Published_At,duration,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,Description,Location,Tags,Latitude,Longitude,Transcript
0,landslide,The Tribune,2021-07-14 04:57:17.998203,Dharamsala flash floods:10 persons missing in ...,Ht6YmGsebdU,2021-07-13T09:39:59Z,PT2M53S,13275,,,,,Read story here: https://tinyurl.com/e2yvdscn\...,NaN,,,,NaN
1,landslide,TV9 Gujarati,2021-07-14 04:57:18.069278,Six houses swept away as landslide hits Himach...,8c8DOhJkIL8,2021-07-13T05:37:53Z,PT38S,3120,,,,,Six houses swept away as landslide hits Himach...,NaN,,,,NaN
2,landslide,TV9 Bharatvarsh,2021-07-14 04:57:18.125585,पहाड़ों पर बरसी आसमानी आफत! Tehri में Landslid...,Mm_kCpnZPdg,2021-07-13T08:43:21Z,PT5M38S,5089,,,,,हिमाचल प्रदेश में लगातार बारिश हो रही है. कई इ...,NaN,,,,NaN
3,landslide,Aaj Tak,2021-07-14 04:57:18.187915,Monsoon Reach At Delhi-NCR I Rain Continue In ...,Dgxdnq7QHmw,2021-07-13T05:21:31Z,PT21M1S,17595,,,,,आखिरकार Delhi-NCR में मानसून पहुंच ही गया. सुब...,NaN,,,,NaN
4,landslide,Oneindia Hindi | वनइंडिया हिन्दी,2021-07-14 04:57:18.239435,Heavy Rain in Kangra: Himachal Pradesh के Boh ...,6UGIhugiF5E,2021-07-13T09:30:00Z,PT3M4S,1813,,,,,Landslide occurred after heavy rains in Boh ar...,NaN,,,,NaN


In [ ]:
import datetime
x = datetime.datetime.now()
#print(x.strftime("%B%d"))
output_date = f'Youtube_General_{x.strftime("%B%d")}.csv'

In [ ]:
ddf2.to_csv(f'/datasets/outputyoutube/Landslide_Project/{output_date}')
print("The final youtube data is saved as : "+output_date)

The final youtube data is saved as : Youtube_General_July14.csv


In [ ]:
!ls /datasets/outputyoutube/Landslide_Project

Cleaning_Clustering_Youtube_Landslide_Prediction.ipynb
config_.json
India_Locations_Villages
Scraping_Youtube_Twitter.ipynb
Twitter_General_July01.csv
Twitter_General_July02.csv
Twitter_General_July03.csv
Twitter_General_July04.csv
Twitter_General_July05.csv
Twitter_General_July06.csv
Twitter_General_July07.csv
Twitter_General_July08.csv
Twitter_General_July09.csv
Twitter_General_June13.csv
Twitter_General_June15.csv
Twitter_General_June16.csv
Twitter_General_June17.csv
Twitter_General_June18.csv
Twitter_General_June19.csv
Twitter_General_June20.csv
Twitter_General_June21.csv
Twitter_General_June22.csv
Twitter_General_June23.csv
Twitter_General_June24.csv
Twitter_General_June25.csv
Twitter_General_June26.csv
Twitter_General_June27.csv
Twitter_General_June28.csv
Twitter_General_June29.csv
Twitter_General_June30.csv
YouTube
Youtube_General_July01.csv
Youtube_General_July02.csv
Youtube_General_July03.csv
Youtube_General_July04.csv
Youtube_General_July05.csv
Youtube_General_July06.csv
Yout

## Focused Crawling

This is just an example of focused crawling

In [ ]:
foc_keywords =json_object["Focused-Crawling"]["Specific-Keywords"]
foc_Keywords=list(foc_keywords)[0]
keys = f"{foc_Keywords}"
foc_keywords=foc_Keywords.split(',')
foc_keywords

['southern-monsoon', 'flood', 'rockfall']

In [ ]:
published_After =json_object['Focused-Crawling']['Specific-Keywords'][keys]['Start_Datetime']

In [ ]:
column_names=["Topic", "Channel_Name" ,"TimeQueried", "Video_Title", "Video_ID", "Published_At", "duration", "viewCount", "likeCount", "dislikeCount",
      "favoriteCount","commentCount", "Description","Location","Tags","latitude","longitude"]
df_youtube_focused=pd.DataFrame(columns=column_names)
textlist_foc=[]
for Query in foc_keywords:
  videos=[]
  next_page_token = None
  while 1:
    search_response = youtube.search().list(q=Query,type="video", part="id,snippet", maxResults=100,regionCode="IN",order="relevance",publishedAfter=published_After).execute() 
    videos+=search_response['items']
    #next_page_token = search_response.get('nextPageToken')
    #print(next_page_token)
    if next_page_token is None:
            break
  count=0
  video_ids=[]
  for video in videos:
      video_ids.append(video['id']['videoId'])
      count=count+1
  count 
  #text_df = pd.DataFrame(textlist,columns=["Transcript"])
  for video_id in video_ids:
      templist=[]
      try:
        responses = YouTubeTranscriptApi.get_transcript(
            video_id, languages=['en'])
        #responses2=responses.get_transcript
        for response in responses:
            text = response['text']
            templist.append(text)
        textlist_foc.append(templist)
      except Exception as e:
        textlist_foc.append("NaN") # Generally when transcript are not available or too many requests are there

  for k in range(len(video_ids)):
    request = youtube.videos().list(
            part="statistics,contentDetails,snippet,status,recordingDetails",
            id=video_ids[k]
        )
    response = request.execute()
    try:
        df_youtube_focused = df_youtube_focused.append({"Topic":Query,"Channel_Name":response["items"][0]["snippet"]["channelTitle"],"TimeQueried":dt.datetime.now(), "Video_Title":response["items"][0]["snippet"]["title"], "Video_ID":response["items"][0]["id"], "Published_At":response["items"][0]["snippet"]["publishedAt"], "duration":response["items"][0]["contentDetails"]["duration"], "viewCount":response["items"][0]["statistics"]["viewCount"], "likeCount":response["items"][0]["statistics"]["likeCount"], "dislikeCount":response["items"][0]["statistics"]["dislikeCount"],"favoriteCount":response["items"][0]["statistics"]["favoriteCount"],"commentCount":response["items"][0]["statistics"]["commentCount"], "Description":response["items"][0]["snippet"]["description"],"Location": response["items"][0]["recordingDetails"]["location"],"Tags":response["items"][0]["snippet"]['tags'][0:10],"Latitude":response["items"][0]["recordingDetails"]['location']['latitude'],"Longitude":response["items"][0]["recordingDetails"]['location']['longitude']},ignore_index = True)
    except:
        df_youtube_focused = df_youtube_focused.append({"Topic":Query,"Channel_Name":response["items"][0]["snippet"]["channelTitle"] ,"TimeQueried":dt.datetime.now(), "Video_Title":response["items"][0]["snippet"]["title"], "Video_ID":response["items"][0]["id"], "Published_At":response["items"][0]["snippet"]["publishedAt"], "duration":response["items"][0]["contentDetails"]["duration"], "viewCount":response["items"][0]["statistics"]["viewCount"], "likeCount":"", "dislikeCount":"","favoriteCount":response["items"][0]["statistics"]["favoriteCount"],"commentCount":"", "Description":response["items"][0]["snippet"]["description"],"Tags":"","Latitude":"","Longitude":""},ignore_index = True)


In [ ]:
text_df_foc = pd.DataFrame()
text_df_foc["Transcript"] = textlist_foc

In [ ]:
df_youtube_focused = pd.concat([df_youtube_focused,text_df_foc],axis=1,ignore_index=False,join="outer")

In [ ]:
df_youtube_focused=df_youtube_focused.drop_duplicates(subset=['Video_ID',
                                  'Video_Title'])

In [ ]:
df_youtube_focused.head()

,Topic,Channel_Name,TimeQueried,Video_Title,Video_ID,Published_At,duration,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,Description,Location,Tags,latitude,longitude,Latitude,Longitude,Transcript
0,southern-monsoon,PADMASUTRA,2021-07-14 04:59:45.261767,കർക്കിടകത്തിൽ രണ്ടാഴ്ച ചെയ്യേണ്ട ദേഹരക്ഷ!Ayurv...,LrFb_M3aydY,2021-07-03T09:06:09Z,PT14M38S,6680,347,6,0,117,Karkidakam (month of Aashada/Aadi) is the mont...,"{'latitude': 51.5073509, 'longitude': -0.12775...","[#karkkidakam, #karkkidakakanji, #karkkidakamr...",NaN,NaN,51.507351,-0.127758,"[[Music], foreign, foreign, foreign, um, forei..."
1,southern-monsoon,City Prepping,2021-07-14 04:59:45.363038,Massive Heat Waves: Expect Even Worse (And Wha...,YerhC3e8Qyc,2021-07-11T16:27:02Z,PT18M7S,68492,,,0,,Record-setting heat waves are increasing in fr...,NaN,,NaN,NaN,,,"[heat waves droughts not seen in, centuries cr..."
2,southern-monsoon,Green Dream Project,2021-07-14 04:59:45.393404,Our First Monsoon Storm Hits! | Prepping for S...,EehICCK3l8U,2021-07-10T19:00:03Z,PT15M56S,32366,3108,32,0,257,Our First Monsoon Storm Hits! | Prepping for S...,"{'latitude': 31.828458, 'longitude': -109.9496...","[green dream project, green dream, gdp, off gr...",NaN,NaN,31.828458,-109.949686,"[hey there you're watching the green, dream pr..."
3,southern-monsoon,8 News NOW Las Vegas,2021-07-14 04:59:45.422879,Experts discuss monsoon season outlook in Las ...,akqaRlcO3qM,2021-06-19T06:09:34Z,PT2M17S,26976,,,0,,Experts discuss monsoon season outlook in Las ...,NaN,,NaN,NaN,,,"[it's mid-june and that means of course, monso..."
4,southern-monsoon,12 News,2021-07-14 04:59:45.456914,Monsoon 2021: July 10 update,zljxmjHemQI,2021-07-11T04:23:41Z,PT1H34M58S,8371,,,0,,Team 12 gives us a look at activity throughout...,NaN,,NaN,NaN,,,"[sorry about that, uh sure we can do that yeah..."


In [ ]:
import datetime
y = datetime.datetime.now()
#print(x.strftime("%B%d"))
output_date_foc = f'Youtube_focused_{y.strftime("%B%d")}.csv'

In [ ]:
df_youtube_focused.to_csv(output_date_foc)
print("The final Focused youtube data is saved as : "+output_date)

In [ ]:
# If we want to update the configuration file dynamically we can uncomment this code
#json_object['Focused-Crawling']['Specific-Keywords'][keys]['End_Datetime'].append(y.strftime("%Y-%m-%dT%XZ"))
#json_object['General-Crawling']['General-Keywords'][keys]['End_Datetime'].append(y.strftime("%Y-%m-%dT%XZ"))

In [ ]:
a_file = open("config_.json", "w")
json.dump(json_object,a_file)
a_file.close()

#Collecting data from twitter

In [ ]:
%%capture
!pip install tweepy

In [ ]:
import tweepy
import re
from nltk.tokenize import TweetTokenizer
consumer_key = 'It6qdrfzzKYBT0F3B1IQYnpEG'
consumer_secret = '2UYJ8ys7zQb6z2251xVpMHeloqK09JnnuXyki7LFZ0bTCnIoiy'
access_token = '1364258204473982978-IrTfDSw2ZvA8IkZqaO30BT0xgBMGv2'
access_token_secret = 'cPSxtNQBI1BetT71fbMP2B3Ok41UXS8QI1bmPeYYKUyqp'


In [ ]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

irrelevant_words = ["Politics","pubg mobile","british parliamentary records",
                    "software career oppounity","child labor"]

def TextClean(tweet):
  tweet = tweet.lower()
  tweet = re.sub(r'@[a-z0-9_]\S+' , '', tweet)
  #tweet = re.sub(r'#[a-z0-9_]\S+', '',tweet)
  tweet = re.sub(r'&[a-z0-9_]\S+','',tweet)
  tweet = re.sub(r'0[?!.+,;$%&"]+','',tweet)
  tweet = re.sub(r'rt[\s]+', '', tweet)
  tweet = re.sub(r'\d+', '', tweet)
  tweet = re.sub(r'\$', '', tweet)
  tweet = re.sub(r'rt+', '', tweet)
  tweet = re.sub(r'https?:?\/\/\S+', '', tweet)
  for emoji in emojis.keys():
    tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])

  
  return tweet

In [ ]:
# google api , database of all location of india
import pandas as pd
i=0
limit = 10000
tweets_df=pd.DataFrame(columns=['Datetime','Tweet','Username','Retweets','Followers','Likes','Location'])
for Query in Gen_keywords :
  for tweet in tweepy.Cursor(api.search,q=Query,count=1000,lang='en',order="relevance", publishedAfter=published_after,regionCode="IN",tweet_mode='extended').items() :

      #print("Tweets Downloaded :",i,'/',100,end = '\r')
      tweets_df = tweets_df.append({'Datetime': tweet.created_at,
                                    'Tweet': tweet.full_text,
                                    'Username': tweet.user.screen_name,
                                    'Retweets': tweet.retweet_count,
                                    'Followers': tweet.user.followers_count ,
                                    'Likes': tweet.favorite_count,
                                    'Location' : tweet.user.location} , ignore_index=True)
      i+=1
      if i>=limit :
        break
      else :
        pass
  tweets_df['CleanTweet'] = tweets_df['Tweet'].apply(TextClean)
      #tweets_df.to_csv(key_word + '.csv', encoding = 'utf-8')
      # tweets_df.drop_duplicates(subset=['Tweet'], inplace = True)
      # tweets_df['CleanTweet'] = tweets_df['Tweet'].apply(TextClean)
      # tweet_tokenizer = TweetTokenizer()
      # tweets_df['CleanTweet'] = tweets_df['CleanTweet'].apply(tweet_tokenizer.tokenize) 
      # tweets_df['CleanTweet'] = [', '.join(map(str, token)) for token in tweets_df['CleanTweet']]

In [ ]:
tweets_df=tweets_df.drop_duplicates(subset=['Tweet',
                                            'CleanTweet'])

In [ ]:
import datetime
y = datetime.datetime.now()
#print(x.strftime("%B%d"))
output_date_gen_twitter = f'Twitter_General_{y.strftime("%B%d")}.csv'
tweets_df.to_csv(f'/datasets/outputyoutube/Landslide_Project/{output_date_gen_twitter}')
print("The final General Twitter data is saved as : "+output_date_gen_twitter)

The final General Twitter data is saved as : Twitter_General_July14.csv


In [ ]:
tweets_df.shape

(3232, 8)

In [ ]:
import datetime
# x = datetime.datetime.now()
# print(x.day,x.month)
x = datetime.datetime.now()
print(x.strftime("%B%d"))

July14


In [ ]:
output_date = f'tweets_{x.strftime("%B%d")}.csv'
tweets_df.to_csv(output_date)

In [ ]:
tweets_df["CleanTweet"]

0        "we won the election by a landslide, we won it...
1        by now everyone in government has seen both ne...
2        . new book, landslide, quotes donald j. trump ...
3        latest: uttarakhand landslide: four bodies rec...
4        landslide blocks village main road in loliem\n...
                               ...                        
10003    / the graph also provides us with greater flex...
10004                         icefall mantle, here i come!
10006    you know what? no. im not letting this slide. ...
10008     the pandora’s box of shit people should’ve ke...
10009    immediate response of tapovan #glacierburst wa...
Name: CleanTweet, Length: 3232, dtype: object

In [ ]:
tweets_df["CleanTweet"][3]

'latest: uttarakhand landslide: four bodies recovered\nfour bodies recovered from boh valley till last night, a total of five bodies have been recovered so far. five to six persons still went missing. rescue operation resumed early morning today, said kangra dt comm nipun jindal.'

In [ ]:
cleanlist =tweets_df["CleanTweet"].values.tolist()
tweetlist = list(dict.fromkeys(cleanlist))

# Archives ( OLD CODE )


In [ ]:
# import requests
# import pickle
# import time
# import json
# import csv
# import operator
# import sys


# Wiki_Threshold=0.0

# def get_response(ques,qid,tagme_ent):
# 	tagme_ent[qid]={}
# 	tagme_ent[qid]['ques']=ques
# 	tagme_ent[qid]['spot']=[]
# 	req_string='https://tagme.d4science.org/tagme/tag?lang=en&include_abstract=true&include_categories=true&gcube-token=9dc5f6c0-3040-411b-9687-75ca53249072-843339462&text='+ques#.encode('utf-8')
# 	try:
# 		r = requests.get(req_string)
# 		wiki=r.json()
# 		annotations=wiki['annotations']
# 		for doc in annotations:
# 			if doc['rho']>=Wiki_Threshold:
# 				tagme_ent[qid]['spot'].append((doc['spot'],doc['title'],doc['rho']))
# 	except:
# 		print("TAGME Problem ", ques)
# 	return tagme_ent


In [ ]:
# pd.options.display.max_colwidth=1000
# #ddf.style.set_properties(**{'text-align': 'left'})
# ddfStyler = ddf.style.set_properties(**{'text-align': 'left'})
# ddfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
# for i in range(50):
#   print(ddf["Description"][i])

In [ ]:
# keyword = pd.read_csv('keywords.txt',header=None)
# keywords=keyword[0].values.tolist()

In [ ]:
# keywords = ['flood-India','LandSlip-India']

In [ ]:
# # Parsing the list of General Keywords
# Gen_Keywords =parsed_json["General-Crawling"]["General-Keywords"]
# Gen_Keywords=list(Gen_Keywords)[0]
# keyword=Gen_Keywords.split(',')

In [ ]:
# import datetime
# x = datetime.datetime.now()
# print(x.strftime("%Y-%m-%dT%XZ")) 
# print(x)
# "2020-01-02T00:00:00Z"

In [ ]:
# This is our Old Concept ( Youtube )

# # Understaand and optimize the queries( how queries are written )
# # unique video_id ( to avoid dublicate data)
# # Add support for next page parsing( next page token for next 50 results(if available))
# column_names=["Topic", "TimeQueried", "Video_Title", "Video_ID", "Published_At", "duration", "viewCount", "likeCount", "dislikeCount",
#       "favoriteCount","commentCount", "Description","Location"]
# ddf=pd.DataFrame(columns=column_names)
# # ,pageToken=next_page_token
# video_ids=[]
# for Query in keywords:
#   videos=[]
#   next_page_token = None
#   while 1:
#     search_response = youtube.search().list(q=Query,type="video", part="id,snippet", maxResults=20,regionCode="IN",order="relevance", publishedAfter="2021-02-02T00:00:00Z").execute() 
#     videos+=search_response['items']
#     #next_page_token = search_response.get('nextPageToken')
#     #print(next_page_token)
#     if next_page_token is None:
#             break


#   count=0
#   for video in videos:
#       #video_ids.append(video['items'][i]["id"]["videoId"])
#       video_ids.append(video['id']['videoId'])

#       count=count+1
#   count  
#   #video_ids = list(set(video_ids))

  
#   for k in range(len(video_ids)):
#       request = youtube.videos().list(
#               part="statistics,contentDetails,snippet,status,recordingDetails",
#               id=video_ids[k]
#           )
#       response = request.execute()
#       #print(response["items"][0]["statistics"]["likeCount"])
#       #video_ids = list(set(video_ids))
#       try:
#           ddf = ddf.append({"Topic":Query, "TimeQueried":dt.datetime.now(), "Video_Title":response["items"][0]["snippet"]["title"], "Video_ID":response["items"][0]["id"], "Published_At":response["items"][0]["snippet"]["publishedAt"], "duration":response["items"][0]["contentDetails"]["duration"], "viewCount":response["items"][0]["statistics"]["viewCount"], "likeCount":response["items"][0]["statistics"]["likeCount"], "dislikeCount":response["items"][0]["statistics"]["dislikeCount"],"favoriteCount":response["items"][0]["statistics"]["favoriteCount"],"commentCount":response["items"][0]["statistics"]["commentCount"], "Description":response["items"][0]["snippet"]["description"],"Location": response["items"][0]["recordingDetails"]["location"]},ignore_index = True)
#       except:
#           ddf = ddf.append({"Topic":Query, "TimeQueried":dt.datetime.now(), "Video_Title":response["items"][0]["snippet"]["title"], "Video_ID":response["items"][0]["id"], "Published_At":response["items"][0]["snippet"]["publishedAt"], "duration":response["items"][0]["contentDetails"]["duration"], "viewCount":response["items"][0]["statistics"]["viewCount"], "likeCount":"", "dislikeCount":"","favoriteCount":response["items"][0]["statistics"]["favoriteCount"],"commentCount":"", "Description":response["items"][0]["snippet"]["description"]},ignore_index = True)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f9dfcc21-4f2b-451a-841d-32b6f6f36d94' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>